# <p style="color:dodgerblue">01 Create Data Source</p>
*We will upload a datasource file which represents stop and search statistics performed by the London Metropolitan Police*  

- this notebook creates the following:
  - s3 bucket to:
    - drop datasource files into 
    - used as resource for redshift
  - iam
    - roles
    - policies
  - redshift cluster
    - model management
    - security management
  - secrets manager
    - cluster and database secret credentials
  - includes clean up cells to delete all above  
  
(At least Kernel 3.11.6 - venv if local)
<hr style="border:1px dotted; color:floralwhite">

# <span style="color:deeppink">GETTING STARTED</span>
# Requirements for this Lab (macOS)
- *See <span style="color:gold">Appendix</span> at the bottom of this lab to install macOS requirements, windows requirements will be similar, apart from Homebrew.*  

<hr style="border:1px dotted">
<hr style="border:1px dotted;color:greenyellow">

# <p style="color:greenyellow">Set Up Requirements</p>
- we do these setup cells here because we can then use the vars and clients to clean up resources later without having to run multiple cells if we lose the kernel  
  
-  <span style="color:greenyellow">Please note we use us-west-2 region as Q in QuickSight is not available worldwide yet<span>

- vars

In [ ]:
import boto3
import json
import random

# verify AWS account and store in myAccountNumber
myAccountNumber = boto3.client("sts").get_caller_identity()["Account"]
print('My account number: {}'.format(myAccountNumber))

# region - we use us-west-2 as Q in QuickSight is limited in other reasons
myRegion='us-west-2'
myLabPrefix='doit-quicksight-london-met-'

# bucket - MUST BE A UNIQUE NAME hence the random postfixes
myBucket=myLabPrefix + 'bucket-' + str(random.randint(0, 1000)) + '-' + str(random.randint(0, 1000))

# iam
myRoleRedshiftAttached=myLabPrefix+'redshift-attached-role'
myRoleRedshiftAttachedARN='RETRIEVED BELOW ONCE CREATED'

myRoleRedshiftCopy=myLabPrefix+'redshift-copy-role'
myPolicyRedshiftCopy1=myLabPrefix + 'redshift-copy-policy'

myRoleQuickSight=myLabPrefix + '-service-role'
myPolicyQuickSight1=myLabPrefix + '??-policy'
myRoleQuickSightARN='RETRIEVED BELOW ONCE CREATED'

# Redshift
myDBClusterIdentifier=myLabPrefix + 'cluster'
myRedshiftDB="london-met"
mySecretRedshiftDevLead=myLabPrefix + 'redshift-devlead-secret'
mySecretRedshiftDevLeadARN='RETRIEVED BELOW ONCE CREATED'
mySecretRedshiftMasterARN='RETRIEVED BELOW ONCE CREATED'

# network
myVPC=myLabPrefix + 'redshift-vpc'
mySGRedshift=myLabPrefix + 'redshift-sg'
mySGQuickSight=myLabPrefix + 'quicksight-sg'
mySubnetGroupRedshift=myLabPrefix + 'redshift-subnet-group'

# local client path for resources
myPathForDataSources='/Users/simondavies/Documents/GitHub/labs/quicksight/met-police/resources/datasource/'

# jupypter notebook path for resources if notebook is used in AWS for example
#myPathForDataSources='/home/ec2-user/SageMaker/labs/quicksight/met-police/resources/datasource/'

print ('Done! Move to the next cell ->')

- create required clients

In [ ]:
# s3
s3 = boto3.client('s3', region_name=myRegion)

# ec2 (reqd for networking services)
ec2 = boto3.client('ec2', region_name=myRegion)

# redshift
redshift = boto3.client('redshift', region_name=myRegion)
redshiftData = boto3.client('redshift-data', region_name=myRegion)

# quicksight
quicksight = boto3.client('quicksight', region_name=myRegion)

# iam
iam = boto3.client('iam', region_name=myRegion)

# secrets manager
secrets = boto3.client('secretsmanager', region_name=myRegion)

# logs (cloudwatch)
logs = boto3.client('logs', region_name=myRegion)

# cidr blocks
vpcCIDR = "10.0.0.0/24"
subnetaCIDR="10.0.0.0/25"
subnetbCIDR="10.0.0.128/25"

print ('Done! Move to the next cell ->')

- tags for all services that are created - you can never have too many tags!
  - make sure you have a tagging policy in place

In [ ]:
# define tags added to all services we create
myTags = [
    {"Key": "env", "Value": "non_prod"},
    {"Key": "owner", "Value": myLabPrefix + "lab"},
    {"Key": "project", "Value": myLabPrefix + "bi"},
    {"Key": "author", "Value": "simon"},
]
myTagsDct = {
    "env": "non_prod",
    "owner": myLabPrefix + "lab",
    "project": myLabPrefix + "bi",
    "author": "simon",
}

print ('Done! Move to the next cell ->')

<hr style="border:1px dotted;color:greenyellow">
<hr style="border:1px dotted;color:orchid">

# <p style="color:orchid">Create IAM</p>
- roles and policies for the services to interact with other services

- iam role and policies which is attached to the redshift cluster giving it required permissions

In [ ]:
# no inline policies required

# trust policy for the role
roleTrust = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": [
                    "redshift.amazonaws.com",
                    "redshift-serverless.amazonaws.com",
                    "sagemaker.amazonaws.com"
                ]
            },
            "Action": "sts:AssumeRole"
        }
    ],
}

# create role
role = iam.create_role(
    RoleName=myRoleRedshiftAttached,
    AssumeRolePolicyDocument=json.dumps(roleTrust),
    Description="Service role for Redshift use",
    Tags=[
        *myTags,
    ],
)

# attach policies to role
iam.attach_role_policy(
    RoleName=role["Role"]["RoleName"], 
    PolicyArn='arn:aws:iam::aws:policy/AmazonRedshiftAllCommandsFullAccess'
)

myRoleRedshiftAttachedARN = role['Role']['Arn']

print ('Done! Move to the next cell ->')

- iam role and policy allowing COPY from S3 to Redshift using the COPY command

In [ ]:
# Inline policies
policyJson = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}/*".format(myBucket),
                "arn:aws:s3:::{}".format(myBucket)
            ]
        }
    ]
}

# Create the policy
policy1 = iam.create_policy(
    PolicyName=myPolicyRedshiftCopy1,
    PolicyDocument=json.dumps(policyJson),
    Description="Policy allowing Redshift COPY to use s3",
    Tags=[
        *myTags,
    ],
)


# trust policy for the role
roleTrust = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "redshift.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ],
}

# create role
role = iam.create_role(
    RoleName=myRoleRedshiftCopy,
    AssumeRolePolicyDocument=json.dumps(roleTrust),
    Description="Service role for Redshift use",
    Tags=[
        *myTags,
    ],
)

# attach policies to role
iam.attach_role_policy(
    RoleName=role["Role"]["RoleName"], 
    PolicyArn=policy1["Policy"]["Arn"]
)

myRoleRedshiftCopyARN = role['Role']['Arn']

print ('Done! Move to the next cell ->')

<hr style="border:1px dotted;color:orchid">
<hr style="border:1px dotted;color:crimson">

# <p style="color:crimson">Create S3 Bucket</p>
- defaults used, will use sse-s3 encryption and block public access

In [ ]:
# create bucket
s3.create_bucket(
    Bucket=myBucket, CreateBucketConfiguration={"LocationConstraint": myRegion}
)
s3.put_bucket_tagging(Bucket=myBucket, Tagging={"TagSet": myTags})

# create a "folder" - really keys as S3 is flat
s3.put_object(Bucket=myBucket, Key="datasource/")

print ('Done! Move to the next cell ->')

- upload resource files to s3 that will be used to create the knowledge base with
  - includes metadata file
  - https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base-ds.html#kb-ds-metadata
  - If you're adding metadata to a vector index in an Amazon Aurora database cluster, you must add a column to the table for each metadata attribute in your metadata files before starting ingestion. The metadata attribute values will be written to these columns.

In [ ]:
# upload each file to the S3 bucket
files = [
    {
        's3key': 'datasource/Stops_LDS_Extract_24Months.csv',
        'localpath': '{}Stops_LDS_Extract_24Months.csv'.format(myPathForDataSources)
    }
]

for file in files:
    print ('uploading: {}'.format(file['s3key']))
    s3.upload_file(file['localpath'], myBucket, file['s3key'], ExtraArgs={'StorageClass': 'STANDARD'})
    print ('uploaded: {}'.format(file['s3key']))

print ('Done! Move to the next cell ->')

<hr style="border:1px dotted;color:crimson">
<hr style="border:1px dotted;color:ForestGreen">

# <p style="color:ForestGreen">Create Network</p>
- vpc  
  - /24 is a reasonable size for a small VPC. This gives you 256 IPs, but note the following:
  - The first 3 and last in the IP range is reserved by AWS
  - VPC cidr blocks cannot overlap
  - Each subnet in a vpc must have a netmask block between /28 (16 IPs) and /16 (65536 IPs)
  - RDS typically requires at least 2 subnets if a standby or read replica is provisioned

https://docs.aws.amazon.com/vpc/latest/userguide/vpc-cidr-blocks.html  
https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/USER_VPC.Scenarios.html  
https://mxtoolbox.com/subnetcalculator.aspx

In [ ]:
# create redshift vpc
vpc_redshift = ec2.create_vpc(
    CidrBlock=vpcCIDR,
    TagSpecifications=[
        {
            "ResourceType": "vpc",
            "Tags": [
                *myTags,
                {"Key": "Name", "Value": myVPC},
            ],
        },
    ],
)

print ('Done! Move to the next cell ->')

- 2 private subnets
  - We'll break the /24 of the VPC over 2 subnets of /25 each
  - The first 4 and last in the IP range is reserved by AWS
  - Subnet cidr blocks cannot overlap
  - RDS typically requires 3 subnets

https://docs.aws.amazon.com/vpc/latest/userguide/subnet-sizing.html
https://mxtoolbox.com/subnetcalculator.aspx

In [ ]:
# create vpc-redshift subnets
subnet_a_redshift = ec2.create_subnet(
    CidrBlock=subnetaCIDR,
    AvailabilityZone=myRegion + "a",
    VpcId=vpc_redshift["Vpc"]["VpcId"],
    TagSpecifications=[
        {
            "ResourceType": "subnet",
            "Tags": [
                *myTags,
                {"Key": "Name", "Value": myVPC + "-subnet-a"},
            ],
        },
    ],
)
subnet_b_redshift = ec2.create_subnet(
    CidrBlock=subnetbCIDR,
    AvailabilityZone=myRegion + "b",
    VpcId=vpc_redshift["Vpc"]["VpcId"],
    TagSpecifications=[
        {
            "ResourceType": "subnet",
            "Tags": [
                *myTags,
                {"Key": "Name", "Value": myVPC + "-subnet-b"},
            ],
        },
    ],
)

# create subnet group
subnet_group_redshift = redshift.create_cluster_subnet_group(
    Description="Redshift subnet group",
    ClusterSubnetGroupName=mySubnetGroupRedshift,
    SubnetIds=[
        subnet_a_redshift["Subnet"]["SubnetId"],
        subnet_b_redshift["Subnet"]["SubnetId"],
    ],
    Tags=[
        *myTags,
        {"Key": "Name", "Value": mySubnetGroupRedshift},
    ],
)


print ('Done! Move to the next cell ->')

- redshift security group
  - we need to create this now as we can reference its arn in the inbound and outbound rules of the quicksight sg

In [ ]:
# create redshift security group
sg_redshift = ec2.create_security_group(
    GroupName=mySGRedshift,
    Description="sg for redshift",
    VpcId=vpc_redshift["Vpc"]["VpcId"],
    TagSpecifications=[
        {
            "ResourceType": "security-group",
            "Tags": [
                *myTags,
                {"Key": "Name", "Value": mySGRedshift},
            ],
        },
    ],
)

print ('Done! Move to the next cell ->')

- quicksight security group
  - we need to create this now as we can reference its arn in the inbound and outbound rules of the redshift sg

In [ ]:
# create quicksight security group
sg_quicksight = ec2.create_security_group(
    GroupName=mySGQuickSight,
    Description="sg for quicksight",
    VpcId=vpc_redshift["Vpc"]["VpcId"],
    TagSpecifications=[
        {
            "ResourceType": "security-group",
            "Tags": [
                *myTags,
                {"Key": "Name", "Value": mySGQuickSight},
            ],
        },
    ],
)

print ('Done! Move to the next cell ->')

- rules for redshift

In [ ]:
# create inbound rule allowing quicksight to reach redshift
ec2.authorize_security_group_ingress(
    GroupId=sg_redshift["GroupId"],
    IpPermissions=[
        {
            "FromPort": 5439,
            "ToPort": 5439,
            "IpProtocol": "tcp",
            'UserIdGroupPairs': [
                {
                    'Description': 'allow quicksight to reach redshift',
                    'GroupId': sg_quicksight["GroupId"],
                },
            ],
        },
    ],
)

# create outbound rule allowing redshift to reach quicksight
ec2.authorize_security_group_egress(
    GroupId=sg_redshift["GroupId"],
    IpPermissions=[
        {
            "FromPort": 5439,
            "ToPort": 5439,
            "IpProtocol": "tcp",
            'UserIdGroupPairs': [
                {
                    'Description': 'allow redshift to reach quicksight',
                    'GroupId': sg_quicksight["GroupId"],
                },
            ],
        },
    ],
)

print ('Done! Move to the next cell ->')

- rules for quicksight

In [ ]:
# create inbound rule allowing redshift to return quicksight
ec2.authorize_security_group_ingress(
    GroupId=sg_quicksight["GroupId"],
    IpPermissions=[
        {
            "FromPort": 5439,
            "ToPort": 5439,
            "IpProtocol": "tcp",
            'UserIdGroupPairs': [
                {
                    'Description': 'allow redshift to return to quicksight',
                    'GroupId': sg_redshift["GroupId"],
                },
            ],
        },
    ],
)

print ('Done! Move to the next cell ->')

<hr style="border:1px dotted;color:ForestGreen">
<hr style="border:1px dotted;color:lightskyblue">

# <p style="color:LightSkyBlue">Create Redshift Cluster</p>
- redshift cluster
  - we create a private master node with 2 data nodes
  - we use a single az (multi az does not support dc2)
  - best practice is multi az with a master node and a number of compute nodes  
  
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift.html

In [ ]:
# we create a dc2.large here as we have very small, static datasets
# if you have larger datasets, or expect regular growth, you can change the instance type to something more suitable
# eg ra3 which separates storage and compute for better scaling - especially useful with regular growth
redshift_cluster = redshift.create_cluster(
    ClusterIdentifier=myDBClusterIdentifier,
    DBName=myRedshiftDB,
    NodeType='dc2.large',
    MasterUsername='masteruser',
    ManageMasterPassword=True,
    ClusterSubnetGroupName=subnet_group_redshift["ClusterSubnetGroup"]["ClusterSubnetGroupName"],
    VpcSecurityGroupIds=[sg_redshift["GroupId"]],
    ClusterType='multi-node',
    NumberOfNodes=2,
    PubliclyAccessible=False,
    Encrypted=True,
    IamRoles=[myRoleRedshiftAttachedARN],
    Tags=[
        *myTags,
        {"Key": "Name", "Value": "{}".format(myDBClusterIdentifier)},
    ],
)

# grab the secrets manager secret arn
mySecretRedshiftMasterARN=redshift_cluster['Cluster']['MasterPasswordSecretArn']

print ('Done! Move to the next cell ->')

- Wait for the cluster to finish creating
  - cant create an instance until the cluster is ready
#### <span style="color:deeppink">you can run the following cell multiple times until the status is available and available</span>

In [ ]:
# can take approx 2 mins to create the cluster
cluster=redshift.describe_clusters(ClusterIdentifier=myDBClusterIdentifier)['Clusters'][0]
print('ClusterStatus={}\nClusterAvailabilityStatus (for queries)={}'.format(cluster['ClusterStatus'],cluster['ClusterAvailabilityStatus']))
print('MasterPasswordSecretArn={}'.format(cluster['MasterPasswordSecretArn']))

In [ ]:
# what is the Secrets Manager masteruser secret ARN, we need these credentials if you want to login to the AWS redshift Query Editor
print('mySecretRedshiftMasterARN={}'.format(mySecretRedshiftMasterARN))
print('myRedshiftDB={}'.format(myRedshiftDB))

In [ ]:
# associate another iam role with the cluster which will allow it to COPY from s3
response = redshift.modify_cluster_iam_roles(
    ClusterIdentifier=myDBClusterIdentifier,
    AddIamRoles=[myRoleRedshiftCopyARN]
)

print ('Done! Move to the next cell ->')

<hr style="border:1px dotted;color:lightskyblue">
<hr style="border:1px dotted;color:Coral">

# <p style="color:Coral">Create A Very Simple Redshift Security Model</p>
- user
  - we create a superuser that has full access - suitable for owners or administrators
  - we create a secret to use when authorising api execute statements
  - best practice we would create users for specific roles in groups or assigned roles with grants
  
https://docs.aws.amazon.com/redshift/latest/mgmt/redshift-secrets-manager-integration.html

In [ ]:
# create a secret for a dev lead user
# we randomise a password to use
usernameDevLead = 'dev_lead'
passwordDevLead = secrets.get_random_password(
    PasswordLength=16,
    ExcludeNumbers=False,
    ExcludePunctuation=True,
    ExcludeUppercase=False,
    ExcludeLowercase=False,
    IncludeSpace=False,
    RequireEachIncludedType=True
)

secretString = {
                "engine": "redshift", \
                "dbClusterIdentifier" : myDBClusterIdentifier, \
                'host': cluster['Endpoint']['Address'],
                "username": usernameDevLead, \
                "password": passwordDevLead['RandomPassword'], \
                "dbname": myRedshiftDB, \
                "port": 5439 \
                }

response = secrets.create_secret(
    Name=mySecretRedshiftDevLead,
    Description="stores the credential for the dev lead user who has access to the db created in redshift cluster {}".format(myDBClusterIdentifier),
    SecretString=json.dumps(secretString),
    Tags=[
        *myTags,
        {"Key": "Name", "Value": mySecretRedshiftDevLead},
        {"Key": "Redshift", "Value": "Used by query editor v2 to find this secret when listing secrets to use"},
    ],
)

mySecretRedshiftDevLeadARN = response['ARN']

print ('Done! Move to the next cell ->')

In [ ]:
# create the database user - we must use the redshift master user to create the first user in a security model
# SQL command to create a database user
# https://docs.aws.amazon.com/redshift/latest/dg/r_CREATE_USER.html
# create a super user = CREATEUSER
sql = f"""
CREATE USER {usernameDevLead}
PASSWORD '{passwordDevLead['RandomPassword']}'
CREATEUSER;
"""

# we connect using the secret for the dev lead database user we created previously
execResponse = redshiftData.execute_statement(
    ClusterIdentifier=myDBClusterIdentifier,
    Database=myRedshiftDB,
    SecretArn=mySecretRedshiftMasterARN,
    Sql=sql,
    StatementName='CREATE DEV LEAD SUPER USER'
)

print ('Done! Move to the next cell ->')

#### <span style="color:deeppink">you can run the following cell multiple times until the statement has finished</span>

In [ ]:
statementResponse=redshiftData.describe_statement(Id=execResponse['Id'])
print('Statement Status={}'.format(statementResponse['Status']))
if statementResponse['Status']=='FAILED':
    print('Statement Error={}'.format(statementResponse['Error']))

<hr style="border:1px dotted;color:Coral">
<hr style="border:1px dotted;color:Aquamarine">

# <p style="color:Aquamarine">Redshift Data</p>
- redshift data
  - we can use the redshift data api to execute ddl and dml statements in redshift
  - alternatively you could use the redshift query editor v2 via the redshift aws console (or local client, eg dbeaver)
  
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift-data.html

In [ ]:
# create schema
redshiftSchema = 'met_police'

sql = f"""
CREATE SCHEMA IF NOT EXISTS {redshiftSchema}
AUTHORIZATION {usernameDevLead};
"""

# we connect using the secret for the dev lead database user we created previously
execResponse = redshiftData.execute_statement(
    ClusterIdentifier=myDBClusterIdentifier,
    Database=myRedshiftDB,
    SecretArn=mySecretRedshiftDevLeadARN,
    Sql=sql,
    StatementName='CREATE SCHEMA'
)

print('sql={}'.format(sql))
print ('Done! Move to the next cell ->')

#### <span style="color:deeppink">you can run the following cell multiple times until the statement has finished</span>

In [ ]:
statementResponse=redshiftData.describe_statement(Id=execResponse['Id'])
print('Statement Status={}'.format(statementResponse['Status']))
if statementResponse['Status']=='FAILED':
    print('Statement Error={}'.format(statementResponse['Error']))

In [ ]:
# create table - ENCODE AUTO is on by default
redshiftTable = 'stop_and_search'

sql = f"""
CREATE TABLE IF NOT EXISTS {redshiftSchema}.{redshiftTable} (
    Date DATE,
    MPS_Area VARCHAR(255),
    Borough_of_Stop VARCHAR(255),
    Borough_Code VARCHAR(10),
    Officer_OCU VARCHAR(255),
    Search_Type VARCHAR(255),
    Subject VARCHAR(255),
    Reason_for_Stop VARCHAR(255),
    Outcome VARCHAR(255),
    Outcome_Reason VARCHAR(255),
    Age DECIMAL(5,2),
    Gender VARCHAR(50),
    Ethnic_Appearance VARCHAR(255),
    Ethnic_Appearance_Group VARCHAR(50),
    Self_defined_Ethnicity VARCHAR(255),
    Self_defined_Ethnicity_Group VARCHAR(50),
    Stop_Count INT,
    Source VARCHAR(50)
)
DISTSTYLE AUTO
SORTKEY AUTO;
"""

# we connect using the secret for the dev lead database user we created previously
execResponse = redshiftData.execute_statement(
    ClusterIdentifier=myDBClusterIdentifier,
    Database=myRedshiftDB,
    SecretArn=mySecretRedshiftDevLeadARN,
    Sql=sql,
    StatementName='CREATE TABLE'
)

print('sql={}'.format(sql))
print ('Done! Move to the next cell ->')

#### <span style="color:deeppink">you can run the following cell multiple times until the statement has finished</span>

In [ ]:
statementResponse=redshiftData.describe_statement(Id=execResponse['Id'])
print('Statement Status={}'.format(statementResponse['Status']))
if statementResponse['Status']=='FAILED':
    print('Statement Error={}'.format(statementResponse['Error']))

In [ ]:
myRoleRedshiftCopy

In [ ]:
# COPY data from S3 into redshift

# S3 bucket and file details
s3File = 'datasource/Stops_LDS_Extract_24Months.csv'

# SQL command to copy data from S3 to Redshift
# https://docs.aws.amazon.com/redshift/latest/dg/copy-parameters-data-source-s3.html
sql = f"""
COPY {redshiftSchema}.{redshiftTable}
FROM 's3://{myBucket}/{s3File}'
IAM_ROLE '{myRoleRedshiftCopyARN}'
REGION '{myRegion}'
CSV
IGNOREHEADER 1
DELIMITER ','
ACCEPTINVCHARS;
"""

# we connect using the secret for the dev lead database user we created previously
execResponse = redshiftData.execute_statement(
    ClusterIdentifier=myDBClusterIdentifier,
    Database=myRedshiftDB,
    SecretArn=mySecretRedshiftDevLeadARN,
    Sql=sql,
    StatementName='COPY FROM S3'
)

print('sql={}'.format(sql))
print ('Done! Move to the next cell ->')

#### <span style="color:deeppink">you can run the following cell multiple times until the statement has finished</span>

In [ ]:
statementResponse=redshiftData.describe_statement(Id=execResponse['Id'])
print('Statement Status={}'.format(statementResponse['Status']))
if statementResponse['Status']=='FAILED':
    print('Statement Error={}'.format(statementResponse['Error']))

In [ ]:
# lets check the row count
sql = f"""
SELECT COUNT(*)
FROM {redshiftSchema}.{redshiftTable};
"""

# we connect using the secret for the dev lead database user we created previously
execResponse = redshiftData.execute_statement(
    ClusterIdentifier=myDBClusterIdentifier,
    Database=myRedshiftDB,
    SecretArn=mySecretRedshiftDevLeadARN,
    Sql=sql,
    StatementName='SELECT COUNT'
)

print('sql={}'.format(sql))
print ('Done! Move to the next cell ->')

In [ ]:
statementResponse=redshiftData.describe_statement(Id=execResponse['Id'])
print('Statement Status={}'.format(statementResponse['Status']))
if statementResponse['Status']=='FINISHED':
    resultDataSet = redshiftData.get_statement_result(
        Id=execResponse['Id']
    )
    print ('resultDataSet={}'.format(resultDataSet['Records']))
if statementResponse['Status']=='FAILED':
    print('Statement Error={}'.format(statementResponse['Error']))

<hr style="border:1px dotted;color:Aquamarine">
<hr style="border:1px dotted;color:deeppink">

# <p style="color:deeppink">STACK 01 COMPLETE!</p>

<hr style="border:1px dotted;color:deeppink">
<hr style="border:1px dotted;color:orangered">

# <p style="color:orangered">Clean Up - DO NOT DO THIS IN THIS LAB!!!!!</p>
# <p style="color:orangered">DO NOT RUN THESE UNLESS YOU WANT TO DESTROY EVERYTHING</p>
- If you have lost the Kernel, run the cells contained in the <span style="color:greenyellow">Set Up Requirements<span> section before the cells below

In [ ]:
# delete redshift cluster
redshift.delete_cluster(
    ClusterIdentifier=myDBClusterIdentifier,
    SkipFinalClusterSnapshot=True
)

print ('Done! Move to the next cell ->')

- Wait for the instance to finish deleting
  - cant delete dependencies until finished
#### <span style="color:deeppink">you can run the following cell multiple times until the status is Deleted</span>

In [ ]:
# can take approx 2 mins to create the cluster
try:
    cluster=redshift.describe_clusters(ClusterIdentifier=myDBClusterIdentifier)
    print('ClusterStatus={}'.format(cluster['Clusters'][0]['ClusterStatus']))
except:
    print("Deleted!")

In [ ]:
# delete secrets manager
# warning, if you use ForceDeleteWithoutRecovery in your own projects there is NO recovery
# normally recovery of the secret is available for 7 to 30 days
secrets.delete_secret(
    SecretId=mySecretRedshiftDevLead, 
    ForceDeleteWithoutRecovery=True
)

print ('Done! Move to the next cell ->')

In [ ]:
# delete roles and policies
iam.detach_role_policy(
    RoleName=myRoleRedshiftAttached, PolicyArn='arn:aws:iam::aws:policy/AmazonRedshiftAllCommandsFullAccess'
)
iam.detach_role_policy(
    RoleName=myRoleRedshiftCopy, PolicyArn='arn:aws:iam::{}:policy/{}'.format(myAccountNumber, myPolicyRedshiftCopy1)
)

iam.delete_role(RoleName=myRoleRedshiftAttached)
iam.delete_role(RoleName=myRoleRedshiftCopy)
iam.delete_policy(PolicyArn='arn:aws:iam::{}:policy/{}'.format(myAccountNumber, myPolicyRedshiftCopy1))

print ('Done! Move to the next cell ->')

In [ ]:
# redshift subnet group
response = redshift.delete_cluster_subnet_group(
    ClusterSubnetGroupName=subnet_group_redshift["ClusterSubnetGroup"]["ClusterSubnetGroupName"]
)

print ('Done! Move to the next cell ->')

In [ ]:
# subnets
response = ec2.delete_subnet(SubnetId=subnet_a_redshift["Subnet"]["SubnetId"], DryRun=False)
response = ec2.delete_subnet(SubnetId=subnet_b_redshift["Subnet"]["SubnetId"], DryRun=False)

print ('Done! Move to the next cell ->')

In [ ]:
sg_redshift

In [ ]:
response = ec2.describe_security_group_rules(
    Filters=[
        {
            'Name': 'group-id',
            'Values': ['{}'.format(sg_redshift["GroupId"])]
        },
    ]
)
response

In [ ]:
# security group rules - cant delete security groups because they have a dependency on each other in the rules
# redshift
rules = ec2.describe_security_group_rules(
    Filters=[
        {
            'Name': 'group-id',
            'Values': ['{}'.format(sg_redshift["GroupId"])]
        },
    ]
)

for sgrule in rules['SecurityGroupRules']:
    if sgrule['IsEgress']==True:
        response = ec2.revoke_security_group_egress(
            GroupId=sg_redshift["GroupId"], 
            SecurityGroupRuleIds=[
                sgrule['SecurityGroupRuleId']
            ],
            DryRun=False
        )
    else:
        response = ec2.revoke_security_group_ingress(
            GroupId=sg_redshift["GroupId"], 
            SecurityGroupRuleIds=[
                sgrule['SecurityGroupRuleId']
            ],
            DryRun=False
        )

# quicksight
rules = ec2.describe_security_group_rules(
    Filters=[
        {
            'Name': 'group-id',
            'Values': ['{}'.format(sg_quicksight["GroupId"])]
        },
    ]
)

for sgrule in rules['SecurityGroupRules']:
    if sgrule['IsEgress']==True:
        response = ec2.revoke_security_group_egress(
            GroupId=sg_quicksight["GroupId"], 
            SecurityGroupRuleIds=[
                sgrule['SecurityGroupRuleId']
            ],
            DryRun=False
        )
    else:
        response = ec2.revoke_security_group_ingress(
            GroupId=sg_quicksight["GroupId"], 
            SecurityGroupRuleIds=[
                sgrule['SecurityGroupRuleId']
            ],
            DryRun=False
        )


# security groups
response = ec2.delete_security_group(GroupId=sg_redshift["GroupId"], DryRun=False)
response = ec2.delete_security_group(GroupId=sg_quicksight["GroupId"], DryRun=False)

print ('Done! Move to the next cell ->')

In [ ]:
# vpc
response = ec2.delete_vpc(VpcId=vpc_redshift["Vpc"]["VpcId"], DryRun=False)

print ('Done! Move to the next cell ->')

In [ ]:
# delete s3 bucket
# NOTE WARNING - this will delete all objects in the bucket with NO prompt or confirmation
s3r = boto3.resource('s3')
bucket = s3r.Bucket(myBucket)
bucket.objects.all().delete()

# delete the bucket
response = s3.delete_bucket(Bucket=myBucket)

print ('Done! Move to the next cell ->')

<hr style="border:1px dotted;color:coral">
<hr style="border:1px dotted;color:gold">

# <p style="color:gold">Appendix - Jupyter Install Requirements (macOS)</p>
#### <p style="color:deeppink">- If you are running VSCode on a laptop, follow all steps below, including the following:</p>
  - Credentials to the AWS account this notebook executes in is provided by AWS configure
  - You must already have an IAM user with code (Command Line Interface) access and AWS access keys to be able to use these credentials in AWS configure  
    
  - arn:aws:iam::###########:user/simon-davies-cli created for this lab  

#### <p style="color:deeppink">- If you are running Jupyter inside an AWS Account, you don't need to do anything!</p>

### <p style="color:gold">1. Homebrew</p> 
If you haven't installed Homebrew, you can install it by running the following command here or in the terminal:

In [ ]:
%%bash
sudo /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"

### <p style="color:gold">1.1 Virtual Environments</p> 
- You can create a virtual environment that ensures any libraries you install are restricted to the venv.
  - https://code.visualstudio.com/docs/python/environments
- To enable the virtual environment once you have created it, ensure you open the folder in vs code containing the notebook files, rather than individual notebook files.

1. Open folder containing the notebooks (DO NOT OPEN INDIVIDUAL FILES, OPEN THE FOLDER)
2. View -> Command Palette
3. \>Python: Create Environment
4. Venv Create a '.venv' virtual environment in current workspace
5. Select latest (or appropriate Python, eg 3.12.2 64-bit)
6. Make sure its selected in the Select Kernel drop down
7. If prompted when running the 1st cell, install ipkyKernel
8. Install boto3 as provided in a cell below if required
9. Ignore cell below

In [ ]:
%%bash
sudo /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"

### <p style="color:gold">1.2 Python</p> 
Once Homebrew is installed, you can install Python using the following command  
*check what you have before installing/upgrading*  
*you will need to quit and restart vsCode to use python once installed (or updated)*

In [ ]:
%%bash
python3 --version
which python3

In [ ]:
%%bash
brew install python

### <p style="color:gold">2. boto3 and other Python requirements</p> 
* boto3 must be installed on your client
  * *Boto3 is the Amazon Web Services (AWS) Software Development Kit (SDK) for Python, which allows Python developers to write software that makes use of services like Amazon S3 and Amazon EC2.*
  * https://boto3.amazonaws.com/v1/documentation/api/latest/index.html  
  
*check what you have before installing/upgrading*  

In [ ]:
%%bash
python3 -m pip show boto3

In [ ]:
pip install -U boto3

### <p style="color:gold">3. aws configure</p> 
*Configure aws configure with credentials, and a user that has all of the Bedrock IAM policies required*  
https://docs.aws.amazon.com/bedrock/latest/userguide/security_iam_id-based-policy-examples.html

In [ ]:
%%bash
aws sts get-caller-identity

<hr style="border:1px dotted;color:gold">